In [1]:
# imported mysql.connector to be able to connect the database
import mysql.connector as mysql

# data management imports
import numpy as np
import pandas as pd
import seaborn as sns
import dtale

KeyboardInterrupt: 

In [ ]:
# connect to database
# connecting to local host
mydb = mysql.connect(
    host="localhost",
    port="3306",
    user="root",
    password="ujveujveyj",
    auth_plugin="mysql_native_password",
    database='Berry_nice'
    
)

# select the correct database here
cursor = mydb.cursor()

In [ ]:
def fetch_all():
    # Fetching all data
    cursor.execute('SELECT * FROM berries;')
    # Fetching column names
    columns = cursor.description
    result = []
    # Adds column names and rows to result variable
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = column
        result.append(tmp)

    df = pd.DataFrame(result)
    df

In [ ]:
df_berry = pd.read_csv('berry_data_cleaned.csv')
df_weather = pd.read_csv('weather_data.csv')
df_forest = pd.read_csv('forest_inventory_data.csv')

In [ ]:
df_berry_weather = pd.merge(df_berry, df_weather, on="latlon", how="left")

In [ ]:
df_everything = pd.merge(df_berry_weather, df_forest, on="xy", how="left")

# northern parts/coordinates is missing in the forest inventory
# let's use the nearest northern measurement point for 
# forest inventory data
helper_row = df_everything.iloc[263]
broken_rows = df_everything[df_everything.isna().any(axis=1)]

nan_cols = [i for i in broken_rows.columns if broken_rows[i].isnull().any()]

for col in nan_cols:
    for index, row in broken_rows.iterrows():
        df_everything.loc[index, col] = helper_row[col]

In [ ]:
df_everything.drop(['x', 'y', 'week_number', 'xy', 'latlon', 'feelslikemax', 'feelslikemin', 'feelslike', 'visibility', 'year-month-day_x', 'lat_x', 'lon_y', 'year-month-day_y', 'lohko', 'osite', 
                    'koeala', 'fra', 'laatu', 'kuvio', 'solarradiation', 'solarenergy', 'uvindex', 'berry_y'], axis=1, inplace=True)

In [ ]:
dtale.show(df_everything).open_browser()

In [ ]:
df_everything.columns

Index(['berry_x', 'observation_date', 'main_tree_type',
       'forest_development_class', 'year', 'month', 'day', 'monthday', 'lon_x',
       'lat_y', 'tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'windspeed',
       'winddir', 'pressure', 'cloudcover', 'moonphase', 'koko9', 'koko564',
       'kunta', 'maku', 'vv', 'mluok', 'alar', 'kptylm', 'ojtil', 'pt',
       'jaksot', 'khlk', 'ppa', 'kasru', 'klpm', 'kpituus', 'ika', 'kokru',
       'bm1', 'bm2', 'bm1ma', 'bm2ma', 'bm1ku', 'bm2ku', 'bm1ko', 'bm2ko',
       'bm1ml', 'bm2ml', 'n_l'],
      dtype='object')